# one- vs multi-threading
In some cases instead of making a single thread python program you can use paralleling and speed it up  
Below is a damn simple programm that demonstrates a simple multrithreading app. Lets assume we have some simple task (def doTask()) and we have to launch it N times in a row (amountOftasks). In this task we have random time insleep that represents some work done or waiting a response from somewhere. Lets first launch it in a single thread one by one and then launch them via multithreading lib nad compare time difference!

In [40]:
import threading
import multiprocessing
import time
import random

amountOftasks = 5

def doTask(n):
    sleeptime = round(random.randint(1e4,1e5)/1e4,1)
    time.sleep(sleeptime)
    print("Job {0} done after {1} seconds at {2}, spent so far {3} seconds.".format(n,sleeptime,time.asctime().split(" ")[3],round(time.time()-startedAt,2)))

startedAt = time.time()
print("Start {0} tasks one-by-one in a row at {1}.".format(amountOftasks,time.asctime().split(" ")[3]))
for i in range(amountOftasks): 
    doTask(i)

startedAt = time.time()
print("Start {0} tasks in multithreading at {1}.".format(amountOftasks,time.asctime().split(" ")[3]))
for i in range(amountOftasks): 
    threading.Thread(target=doTask, args=(i,)).start()


Start 5 tasks one-by-one in a row at 01:42:30.
Job 0 done after 5.2 seconds at 01:42:35, spent so far 5.2 seconds.
Job 1 done after 2.0 seconds at 01:42:37, spent so far 7.21 seconds.
Job 2 done after 2.1 seconds at 01:42:39, spent so far 9.31 seconds.
Job 3 done after 6.7 seconds at 01:42:46, spent so far 16.01 seconds.
Job 4 done after 9.2 seconds at 01:42:55, spent so far 25.22 seconds.
Start 5 tasks in multithreading at 01:42:55.


# multithreading is much faster
Yes, in some cases you can significantly speedup your program computing tasks in parallel. Lets rewrite it with more control over threads:

In [41]:
threads = []
for i in range(amountOftasks):
    t = threading.Thread(target=doTask, args=(i,))
    threads.append(t)
    t.start()

# .join() method blocks main thread until this tread would not be killed

startedAt = time.time()
print("Start {0} tasks in multithreading at {1}.".format(amountOftasks,time.asctime().split(" ")[3]))
for one_thread in threads:
     # stdout also blocked here because it is controlled by main thread
    print(".join() thread {0} after {1} seconds.".format(str(one_thread),round(time.time()-startedAt,1)))
    one_thread.join()

print("All tasks are done at {0} seconds.".format(round(time.time()-startedAt,1)))

Start 5 tasks in multithreading at 01:42:55.
.join() thread <Thread(Thread-59, started 140426591577664)> after 0.0 seconds.
Job 1 done after 1.7 seconds at 01:42:57, spent so far 1.66 seconds.
Job 0 done after 2.2 seconds at 01:42:57, spent so far 2.2 seconds.
.join() thread <Thread(Thread-60, started 140426027722304)> after 2.2 seconds.
Job 2 done after 3.2 seconds at 01:42:58, spent so far 3.16 seconds.
Job 4 done after 4.7 seconds at 01:43:00, spent so far 4.7 seconds.
Job 4 done after 5.6 seconds at 01:43:01, spent so far 5.56 seconds.
Job 2 done after 7.0 seconds at 01:43:02, spent so far 7.0 seconds.
Job 3 done after 7.4 seconds at 01:43:02, spent so far 7.36 seconds.
Job 0 done after 7.6 seconds at 01:43:03, spent so far 7.56 seconds.
Job 3 done after 8.6 seconds at 01:43:04, spent so far 8.61 seconds.
Job 1 done after 8.7 seconds at 01:43:04, spent so far 8.71 seconds.
.join() thread <Thread(Thread-61, stopped 140426019329600)> after 8.7 seconds.
.join() thread <Thread(Thread-6

# multiprocessing or multithreading!
lets do the same in multiprocessing instead of multithreading. btw both libraries have very similar sintax

In [42]:

startedAt = time.time()
processes = [ ]
for i in range(amountOftasks):
    t = multiprocessing.Process(target=doTask, args=(i,))
    processes.append(t)
    t.start()

startedAt = time.time()
print("Start {0} tasks in multiprocessing at {1}.".format(amountOftasks,time.asctime().split(" ")[3]))
for one_process in processes:
    print(".join() Process {0} after {1} seconds.".format(str(one_process),round(time.time()-startedAt,1)))
    one_process.join()

print("All tasks are done in {0} seconds.".format(round(time.time()-startedAt,1)))

Start 5 tasks in multiprocessing at 01:43:04.
.join() Process <Process name='Process-36' pid=50261 parent=47625 started> after 0.0 seconds.
Job 2 done after 1.7 seconds at 01:43:05, spent so far 1.71 seconds.
Job 1 done after 2.2 seconds at 01:43:06, spent so far 2.21 seconds.
Job 4 done after 3.8 seconds at 01:43:08, spent so far 3.82 seconds.
Job 0 done after 4.8 seconds at 01:43:09, spent so far 4.81 seconds.
.join() Process <Process name='Process-37' pid=50262 parent=47625 stopped exitcode=0> after 4.8 seconds.
.join() Process <Process name='Process-38' pid=50263 parent=47625 stopped exitcode=0> after 4.8 seconds.
.join() Process <Process name='Process-39' pid=50264 parent=47625 started> after 4.8 seconds.
Job 3 done after 8.1 seconds at 01:43:12, spent so far 8.12 seconds.
.join() Process <Process name='Process-40' pid=50265 parent=47625 stopped exitcode=0> after 8.1 seconds.
All tasks are done in 8.1 seconds.


# interprocess communications 1
Any form of data exchange between processes is called interprocess communication. This can be done with shared memory by multiple processes or with data passing between them. Lets look first on memory sharing by a multithreading:

In [43]:
# some shared lins to be modifyed by several threads
mylist = [ ]
print("Shared list is now empty! len: {0}; content: {1}".format(len(mylist),str(mylist)))
def doTask(n):
    sleeptime = round(random.randint(1e4,1e5)/1e4,1)
    time.sleep(sleeptime)
    mylist.append(threading.get_ident())   # race condition aware!
    print("Job (add thread inent to shared list) {0} done after {1} seconds at {2}, spent so far {3} seconds.".format(n,sleeptime,time.asctime().split(" ")[3],round(time.time()-startedAt,2)))

threads = []
for i in range(amountOftasks):
    t = threading.Thread(target=doTask, args=(i,))
    threads.append(t)
    t.start()

startedAt = time.time()
print("Start {0} tasks in multithreading at {1}.".format(amountOftasks,time.asctime().split(" ")[3]))
for one_thread in threads:
    print(".join() thread {0} after {1} seconds.".format(str(one_thread),round(time.time()-startedAt,1)))
    one_thread.join()
    
print("All tasks are done in {0} seconds.".format(round(time.time()-startedAt,1)))
print("Shared list is not empty enymore! len: {0}; content: {1}".format(len(mylist),str(mylist)))

Shared list is now empty! len: 0; content: []
Start 5 tasks in multithreading at 01:43:12.
.join() thread <Thread(Thread-64, started 140426027722304)> after 0.0 seconds.
Job (add thread inent to shared list) 1 done after 3.6 seconds at 01:43:16, spent so far 3.6 seconds.
Job (add thread inent to shared list) 0 done after 3.8 seconds at 01:43:16, spent so far 3.8 seconds.
.join() thread <Thread(Thread-65, stopped 140426010936896)> after 3.8 seconds.
.join() thread <Thread(Thread-66, started 140425994151488)> after 3.8 seconds.
Job (add thread inent to shared list) 3 done after 4.2 seconds at 01:43:16, spent so far 4.2 seconds.
Job (add thread inent to shared list) 2 done after 5.2 seconds at 01:43:17, spent so far 5.2 seconds.
.join() thread <Thread(Thread-67, stopped 140426036115008)> after 5.2 seconds.
.join() thread <Thread(Thread-68, started 140426636949056)> after 5.2 seconds.
Job (add thread inent to shared list) 4 done after 5.6 seconds at 01:43:18, spent so far 5.6 seconds.
All 

# interprocess communications 2
If we will do the same with multiprocessing, the result wont be the same, because multiprocessing dont use shared memory and global variable will be copied to memory of each process before it wil be changed and wont be rewritten after procccess will finish its job.

In [44]:

mylist = [ ]
print("Shared list is now empty! len: {0}; content: {1}".format(len(mylist),str(mylist)))

startedAt = time.time()
processes = [ ]
for i in range(amountOftasks):
    t = multiprocessing.Process(target=doTask, args=(i,))
    processes.append(t)
    t.start()

startedAt = time.time()
print("Start {0} tasks in multiprocessing at {1}.".format(amountOftasks,time.asctime().split(" ")[3]))
for one_process in processes:
    print(".join() Process {0} after {1} seconds.".format(str(one_process),round(time.time()-startedAt,1)))
    one_process.join()

print("All tasks are done in {0} seconds.".format(round(time.time()-startedAt,1)))
print("Shared list is still empty, because we use multiprocessing! len: {0}; content: {1}".format(len(mylist),str(mylist)))


Shared list is now empty! len: 0; content: []
Start 5 tasks in multiprocessing at 01:43:18.
.join() Process <Process name='Process-41' pid=50294 parent=47625 started> after 0.0 seconds.
Job (add thread inent to shared list) 0 done after 1.1 seconds at 01:43:19, spent so far 1.11 seconds.
.join() Process <Process name='Process-42' pid=50295 parent=47625 started> after 1.1 seconds.
Job (add thread inent to shared list) 4 done after 1.9 seconds at 01:43:19, spent so far 1.92 seconds.


# interprocess communications 3
How to communicate between processes if we don't have an any shared memory? We can use fifo pipes between processes for this reason:

In [ ]:
q = multiprocessing.Queue()

def doTask(n):
    sleeptime = round(random.randint(1e4,1e5)/1e4,1)
    time.sleep(sleeptime)
#    mylist.append(threading.get_ident())   # race condition aware!
    q.put(os.getpid())
    print("Job (add thread inent to shared list) {0} done after {1} seconds at {2}, spent so far {3} seconds.".format(n,sleeptime,time.asctime().split(" ")[3],round(time.time()-startedAt,2)))

mylist = [ ]
print("Shared list is now empty! len: {0}; content: {1}".format(len(mylist),str(mylist)))

startedAt = time.time()
processes = [ ]
for i in range(amountOftasks):
    t = multiprocessing.Process(target=doTask, args=(i,))
    processes.append(t)
    t.start()

startedAt = time.time()
print("Start {0} tasks in multiprocessing at {1}.".format(amountOftasks,time.asctime().split(" ")[3]))
for one_process in processes:
    print(".join() Process {0} after {1} seconds.".format(str(one_process),round(time.time()-startedAt,1)))
    one_process.join()


while not q.empty():
    mylist.append(q.get())

print("All tasks are done in {0} seconds.".format(round(time.time()-startedAt,1)))
print("Shared list is not empty now! len: {0}; content: {1}".format(len(mylist),str(mylist)))


# daemon and non daemon processes 1:
difference between daemon and non-daemon threads:
* daemon thread can be killed at the end of the main thread before end of damon thread
* non-daemon thread would make main thread to wait until non-daemon threads would end before exit() on main thread

In [ ]:
def doTask(n):
	print("Job {0} started at {1}.".format(n,time.asctime().split(" ")[3]))
	sleeptime = round(random.randint(1e4,1e5)/1e4,1)
	time.sleep(sleeptime)
	print("Job {0} done after {1} seconds at {2}, spent so far {3} seconds.".format(n,sleeptime,time.asctime().split(" ")[3],round(time.time()-startedAt,2)))

t = threading.Thread(name='non-daemon', target=doTask, args=('non-daemon',))
d = threading.Thread(name='daemon', target=doTask, args=('daemon',))
d.setDaemon(True)

startedAt = time.time()
d.start()
t.start()



# daemon and non daemon processes 2: 
use .join() to wait until daemon thread would be ended

In [ ]:
t = threading.Thread(name='non-daemon', target=doTask, args=('non-daemon',))
d = threading.Thread(name='daemon', target=doTask, args=('daemon',))
d.setDaemon(True)

d.start()
t.start()

d.join()
t.join()


# exmpl
see python/multiprocessing example provided, you can use it as a start point for the making an app